In [13]:
import tensorflow as tf
from tqdm import tqdm
import copy
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [14]:
clients = [1, 2, 3]
epochs = 20

In [15]:
import tensorflow as tf
import numpy as np

# Load the MNIST dataset
(x_train_all, y_train_all), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocess the data
x_train_all = x_train_all.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

In [16]:
from sklearn.model_selection import train_test_split

# assume X is your feature data and y is your target data
X_train, X_test, y_train, y_test = train_test_split(
    x_train_all, y_train_all, test_size=0.2, random_state=42
)

# split data into n parts
n_parts = len(clients)
part_size = len(X_train) // n_parts
dataset_parts = []
for i in range(n_parts):
    start = i * part_size
    end = (i + 1) * part_size
    X_part = X_train[start:end]
    y_part = y_train[start:end]
    dataset_parts.append((X_part, y_part))

In [17]:
def model_init():
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(6, activation="relu"),
            tf.keras.layers.Dense(10, activation="softmax"),
        ]
    )
    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [18]:
models = []
for _ in range(len(clients)):
    models.append(model_init())

In [19]:
import numpy as np
from Pyfhel import Pyfhel

HE = Pyfhel()  # Creating empty Pyfhel object
ckks_params = {
    "scheme": "CKKS",  # can also be 'ckks'
    "n": 2**14,  # Polynomial modulus degree. For CKKS, n/2 values can be
    #  encoded in a single ciphertext.
    #  Typ. 2^D for D in [10, 15]
    "scale": 2**30,  # All the encodings will use it for float->fixed point
    #  conversion: x_fix = round(x_float * scale)
    #  You can use this as default scale or use a different
    #  scale on each operation (set in HE.encryptFrac)
    "qi_sizes": [60, 30, 30, 30, 60],  # Number of bits of each prime in the chain.
    # Intermediate values should be  close to log2(scale)
    # for each operation, to have small rounding errors.
}
HE.contextGen(**ckks_params)  # Generate context for ckks scheme
HE.keyGen()  # Key Generation: generates a pair of public/secret keys
HE.rotateKeyGen()

In [20]:
shapedims = [l.shape for l in models[0].get_weights()]
print(shapedims)

[(784, 6), (6,), (6, 10), (10,)]


In [21]:
def encrypt_wt(wtarray):
    cwt = []
    for layer in wtarray:
        flat_array = layer.astype(np.float64).flatten()

        chunks = np.array_split(flat_array, (len(flat_array) + 2**10 - 1) // 2**10)
        clayer = []
        for chunk in chunks:
            ptxt = HE.encodeFrac(chunk)
            ctxt = HE.encryptPtxt(ptxt)
            clayer.append(ctxt)
        cwt.append(clayer.copy())
    return cwt
        

In [22]:
def aggregate_wt(cwts):
    resmodel = []
    for j in range(len(cwts[0])):  # for layers
        layer = []
        for k in range(len(cwts[0][j])):  # for chunks
            tmp = cwts[0][j][k].copy()
            for i in range(1, len(cwts)):  # for clients
                tmp = tmp + cwts[i][j][k]
            tmp = tmp / len(cwts)
            layer.append(tmp)
        resmodel.append(layer)

    res = [resmodel.copy() for _ in range(len(clients))]
    return res
    

In [23]:
def decrypt_weights(res):
    decrypted_weights = []
    for client_weights, model in zip(res, models):
        decrypted_client_weights = []
        wtarray = model.get_weights()
        for layer_weights, layer in zip(client_weights, wtarray):
            decrypted_layer_weights = []
            flat_array = layer.astype(np.float64).flatten()
            chunks = np.array_split(flat_array, (len(flat_array) + 2**10 - 1) // 2**10)
            for chunk, encrypted_chunk in zip(chunks, layer_weights):
                decrypted_chunk = HE.decryptFrac(encrypted_chunk)
                original_chunk_size = len(chunk)
                decrypted_chunk = decrypted_chunk[:original_chunk_size]
                decrypted_layer_weights.append(decrypted_chunk)
            decrypted_layer_weights = np.concatenate(decrypted_layer_weights, axis=0)
            decrypted_layer_weights = decrypted_layer_weights.reshape(layer.shape)
            decrypted_client_weights.append(decrypted_layer_weights)
        decrypted_weights.append(decrypted_client_weights)
    return decrypted_weights

In [24]:
cwts = [encrypt_wt(model.get_weights()) for model in models]
for e in tqdm(range(epochs)):
    wts = decrypt_weights(cwts)
    cwts = []
    for wt,model , dataset in zip(wts, models, dataset_parts):
        model.set_weights(wt)
        model.fit(dataset[0], dataset[1], epochs=1,  verbose=1)
        wtarray = model.get_weights()
        cwts.append(encrypt_wt(wtarray))

    cwts = aggregate_wt(cwts)

  0%|          | 0/20 [00:00<?, ?it/s]

500/500 [==============================] - 1s 724us/step - loss: 1.2351 - accuracy: 0.6155


  5%|▌         | 1/20 [00:02<00:45,  2.40s/it]

500/500 [==============================] - 0s 773us/step - loss: 0.7648 - accuracy: 0.8060


 10%|█         | 2/20 [00:04<00:35,  1.97s/it]

...

 90%|█████████ | 18/20 [00:31<00:03,  1.75s/it]

500/500 [==============================] - 0s 797us/step - loss: 0.3241 - accuracy: 0.9082


 95%|█████████▌| 19/20 [00:33<00:01,  1.76s/it]

500/500 [==============================] - 0s 799us/step - loss: 0.3222 - accuracy: 0.9086


100%|██████████| 20/20 [00:35<00:00,  1.77s/it]
